In [1]:
import pandas as pd

In [2]:
states_df = pd.read_csv("States Data/22.csv")

In [3]:
#Checking for null values in dataset
print(states_df.isnull().values.any())

False


There are no missing values in the dataset.

In [4]:
#Calculating the difference between each row as cumulative data is given in the states dataset. 
states_df_inter = states_df.set_index('Date').diff()
states_df_inter.fillna(0, inplace=True)

In [5]:
#Calculating the Q1, Q3 and IQR
Q1 = states_df_inter.quantile(0.25)
Q3 = states_df_inter.quantile(0.75)
IQR = Q3 - Q1

print(Q1 - 1.5 * IQR)
print(Q3 + 1.5 * IQR)

TN confirmed    -2696.750
TX confirmed   -10068.125
TN deaths         -45.000
TX deaths        -213.000
dtype: float64
TN confirmed     5275.250
TX confirmed    19458.875
TN deaths          83.000
TX deaths         401.000
dtype: float64


In [6]:
states_df_inter.shape

(438, 4)

In [7]:
states_df_inter.to_csv("States Data/22_inter.csv")

In [12]:
#Getting the rows which are outliers (detected by Tukey's rule with alpha = 1.5)
outliers = states_df_inter[((states_df_inter < (Q1 - 1.5 * IQR)) |(states_df_inter > (Q3 + 1.5 * IQR))).any(axis=1)]
outliers

,Date,TN confirmed,TX confirmed,TN deaths,TX deaths
187,2020-07-27,2547.0,4412.0,11.0,673.0
191,2020-07-31,5793.0,8360.0,27.0,295.0
292,2020-11-09,5860.0,4193.0,15.0,22.0
294,2020-11-11,3630.0,11071.0,89.0,141.0
298,2020-11-15,5817.0,6008.0,16.0,89.0
...,...,...,...,...,...
378,2021-02-03,1856.0,17372.0,133.0,418.0
379,2021-02-04,3154.0,15131.0,169.0,439.0
380,2021-02-05,2661.0,15357.0,203.0,401.0
383,2021-02-08,1203.0,5987.0,96.0,57.0


In [9]:
outliers.to_csv("States Data/outliers.csv")
outliers_date = outliers.reset_index().iloc[:,0].tolist()

There were 62 outliers. The outlier values can be found in outliers.csv

In [10]:
#Getting the cumulative data back again from per day stats
states_df_inter = states_df_inter.reset_index()
states_df_out = states_df_inter[~states_df_inter['Date'].isin(outliers_date)]
states_df_out.set_index('Date', inplace = True)
states_df_out = states_df_out.cumsum(axis = 0)
states_df_out.shape

(376, 4)

In [11]:
states_df_out.to_csv("States Data/22_cleaned.csv")

The outliers are dropped and the remaining values are stored in 22_cleaned.csv